In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setAppName('SemiJoin').setMaster('yarn')\
   .set('spark.sql.legacy.timeParserPolicy', 'LEGACY')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

In [3]:
emp = [(1, 'Anna'), (2, 'Bill'), (3, 'John'), (4, 'Kate')]
 
df_emp = spark.createDataFrame(data=emp, schema=['emp_id', 'name'])
 
empHistPos = [ (1, 'Jr Developer', 'PROMOTED'),
               (1, 'Developer', 'PROMOTED'),
               (1, 'Sr Developer', ''),
                  (2, 'Jr Manager', ''),
               (3, 'Jr Sales Repr', 'PROMOTED'),
               (3, 'Sales Repr', 'PROMOTED'),
               (3, 'Sr Sales Repr', ''),
                  (4, 'Jr HR Manager', 'PROMOTED'),
                  (4, 'HR Manager', 'FIRED')]
 
df_hist_pos = spark.createDataFrame(data=empHistPos, schema=['emp_id', 'pos_name', 'change_reason'])
 
df_promoted = df_hist_pos.filter("change_reason = 'PROMOTED'") # double records
 
df_emp_promoted = df_emp.join(df_promoted, 'emp_id', 'leftsemi')
 
df_emp_promoted.orderBy('emp_id').show(truncate=False)

+------+----+
|emp_id|name|
+------+----+
|1     |Anna|
|3     |John|
|4     |Kate|
+------+----+



In [4]:
spark.stop()